In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Import necessary libraries
from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Load and preprocess data with proper handling
train_df = pd.read_csv("train-data.tsv", sep='\t', header=None, names=["label", "text"], on_bad_lines='skip')
test_df = pd.read_csv("valid-data.tsv", sep='\t', header=None, names=["label", "text"], on_bad_lines='skip')

# Clean and prepare data
train_df = train_df.dropna().reset_index(drop=True)
test_df = test_df.dropna().reset_index(drop=True)

# Combine datasets for better vectorization
full_df = pd.concat([train_df, test_df])
train_df["label"] = train_df["label"].map({"ham": 0, "spam": 1})
test_df["label"] = test_df["label"].map({"ham": 0, "spam": 1})

# Create TextVectorization layer with proper settings
max_features = 5000
sequence_length = 50

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length,
    standardize='lower_and_strip_punctuation',
    pad_to_max_tokens=False
)

# Adapt vectorization layer to all text data
vectorize_layer.adapt(full_df["text"].values)

# Build optimized model architecture
model = Sequential([
    layers.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    layers.Embedding(
        input_dim=max_features + 1,  # +1 for OOV tokens
        output_dim=64,
        mask_zero=True
    ),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# Compile model with tuned parameters
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    metrics=['accuracy']
)

# Prepare datasets with proper batching
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_df["text"].values, train_df["label"].values)
).shuffle(len(train_df)).batch(128).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df["text"].values, test_df["label"].values)
).batch(128).prefetch(tf.data.AUTOTUNE)

# Train with early stopping and validation
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=30,
    callbacks=[early_stop],
    verbose=0
)

# Final prediction function
def predict_message(pred_text):
    # Convert to Tensor and add batch dimension
    input_tensor = tf.convert_to_tensor([pred_text])
    # Get prediction
    prediction = model.predict(input_tensor, verbose=0)
    # Extract probability of spam
    prob_spam = float(prediction[0][0])
    # Determine label based on threshold
    label = "spam" if prob_spam >= 0.5 else "ham"
    # Return probability and label
    return [prob_spam, label]

# Test prediction
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
